*This File is For Traing and Building our Model(MobileNet Model) on our own Face images Data set which is an Example of Transfer Learning*

In [1]:
#Importing MobileNet Pre-build model to do Fine Tunning and Transfer learning on.
from keras.applications import MobileNet

Using TensorFlow backend.


In [2]:
#Taking Model Data from imagenet.
model = MobileNet(weights='imagenet', input_shape=((224,224,3)),include_top=False)

In [3]:
#specifing Model layers
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [4]:
#Freezing all Layers(Making them all Non-Trainable)
for layers in model.layers:
    layers.trainable=False

In [5]:
model.layers[0].input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [6]:
from keras.models import Sequential

In [7]:
from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D

In [8]:
#saving output layer(last Fully Connected layer) in top_model Variable for Adding a new layer before it.
top_model = model.output

In [9]:
#creating a New fully Connected Layer at second lasrt place
top_model = GlobalAveragePooling2D()(top_model)

In [10]:
top_model = Dense(units=1024,activation='relu')(top_model)

In [11]:
top_model = Dense(units=512,activation='relu')(top_model)

In [12]:
top_model = Dense(units=2,activation='softmax')(top_model)

In [13]:
from keras.models import Model

In [14]:
#adding up old model and the new layer we just added in new_model Variable
new_model = Model(inputs=model.input,outputs=top_model)

In [15]:
new_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [16]:
from keras.preprocessing.image import ImageDataGenerator

In [17]:
#locations of Test and Train Data sets which we Have taken by us using Haarcascade.
train_data_dir = 'C:\Users\a\Desktop\Github\Face Recognization using TL\Dataset\Training_data'
test_data_dir = 'C:\Users\a\Desktop\Github\Face Recognization using TL\Dataset\Test_data'

# Let's use some data augmentaiton(increaing Quantity of Data for Model Training)
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (can be 16-32)
batch_size = 32

#Genrating Dataset
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224,224),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        test_data_dir,
        target_size=(224,224),
        batch_size=batch_size,
        class_mode='categorical')

Found 2000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [18]:
from keras.optimizers import RMSprop

In [19]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [20]:
#Saving our Model in .h5 extension file
checkpoint = ModelCheckpoint("MobileNet with TL and FT.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Compiling our Model here using Loss,Optimizer and Metrics Accuracy
new_model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])


# We only train 1 EPOCHS(as it requires Large Computing Power) 
epochs = 1
batch_size = 32

history = new_model.fit_generator(
    train_generator,
    steps_per_epoch = 2000,
    epochs = 1,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = 400)

Epoch 1/1
62/62 [==============================] - 347s 6s/step - loss: 0.7191 - accuracy: 0.9202 - val_loss: 3.8159 - val_accuracy: 0.5052

Epoch 00001: val_loss improved from inf to 3.81594, saving model to MobileNet with TL and FT.h5


In [3]:
print('Your Model is Trained with Transfer Learning')

Your Model is Trained with Transfer Learning


In [2]:
print('Your Model is Saved Finally!')

Your Model is Saved Finally!
